In [22]:
import PyPDF2
import re
import smtplib
import os
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication

def extract_hr_details_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        
        text = ""
        for page in reader.pages:
            text += page.extract_text() + "\n"
            
        print(len(text))
        print("Extracted text from PDF (first 500 characters):")
        print(text[167400:167453] + "..." if len(text) > 500 else text)
        
        hr_details = []
        
        email_pattern = r'[\w\.-]+@[\w\.-]+\.\w+'
        emails = re.findall(email_pattern, text)
        
        if emails:
            print(f"Found {len(emails)} email addresses. Extracting surrounding data...")
            
            for email in emails:
                email_pos = text.find(email)
                
                context_start = max(0, email_pos - 100)
                context_end = min(len(text), email_pos + len(email) + 100)
                context = text[context_start:context_end]
                
                context_lines = [line.strip() for line in context.split('\n') if line.strip()]
                
                email_line_idx = -1
                for i, line in enumerate(context_lines):
                    if email in line:
                        email_line_idx = i
                        break
                
                name = ""
                title = ""
                company = ""
                
                if email_line_idx >= 0:
                    for i in range(max(0, email_line_idx - 2), email_line_idx):
                        line = context_lines[i]
                        if 3 <= len(line) <= 40 and any(c.isalpha() for c in line) and '@' not in line:
                            name = line
                            break
                    
                    if not name and email_line_idx < len(context_lines):
                        line = context_lines[email_line_idx]
                        name_part = line.split(email)[0].strip()
                        if len(name_part) > 0 and len(name_part) < 40:
                            name = name_part
                    
                    for i in range(email_line_idx + 1, min(len(context_lines), email_line_idx + 3)):
                        line = context_lines[i]
                        title_keywords = ["director", "hr", "head", "manager", "chief", "vp", "vice president", 
                                         "talent", "human resource", "recruitment", "people"]
                        if any(keyword in line.lower() for keyword in title_keywords) and '@' not in line:
                            title = line
                            break
                    
                    start_idx = email_line_idx + 1 if not title else email_line_idx + 2
                    for i in range(start_idx, min(len(context_lines), email_line_idx + 4)):
                        line = context_lines[i]
                        if any(keyword in line.lower() for keyword in title_keywords) or '@' in line:
                            continue
                        if 3 <= len(line) <= 50 and any(c.isalpha() for c in line):
                            company = line
                            break
                
                if not name or not title or not company:
                    if not company:
                        domain = email.split('@')[1]
                        company_name = domain.split('.')[0]
                        company = company_name.replace('-', ' ').replace('_', ' ').title()
                    
                    if not name:
                        local_part = email.split('@')[0]
                        name_parts = re.split(r'[._-]', local_part)
                        name = ' '.join(part.title() for part in name_parts)
                    
                    if not title:
                        title = "HR Professional"
                
                hr_details.append({
                    'name': name,
                    'email': email,
                    'title': title,
                    'company': company
                })
        
        if not hr_details:
            print("Trying to extract data from potential table structure...")
            
            headers_pattern = r'(Name|Email|Title|Company|Position|Organization)'
            if re.search(headers_pattern, text, re.IGNORECASE):
                lines = text.split('\n')
                
                header_idx = -1
                for i, line in enumerate(lines):
                    if re.search(headers_pattern, line, re.IGNORECASE) and len(re.findall(headers_pattern, line, re.IGNORECASE)) >= 3:
                        header_idx = i
                        break
                
                if header_idx >= 0:
                    for i in range(header_idx + 1, len(lines)):
                        line = lines[i].strip()
                        if not line or len(line) < 10:
                            continue
                            
                        email_match = re.search(email_pattern, line)
                        if email_match:
                            email = email_match.group(0)
                            
                            parts = re.split(r'\s{2,}|\t|,', line)
                            
                            parts = [p for p in parts if email not in p]
                            
                            name = parts[0] if len(parts) > 0 else ""
                            title = parts[1] if len(parts) > 1 else "HR Professional"
                            company = parts[2] if len(parts) > 2 else ""
                            
                            hr_details.append({
                                'name': name,
                                'email': email,
                                'title': title,
                                'company': company
                            })
        
        cleaned_hr_details = []
        for hr in hr_details:
            if not hr['email'] or '@' not in hr['email']:
                continue
                
            name = re.sub(r'[^a-zA-Z\s\.-]', '', hr['name']).strip()
            if name:
                hr['name'] = name
                
            if not hr['title'] or len(hr['title']) < 3:
                hr['title'] = "HR Professional"
                
            if not hr['company'] or len(hr['company']) < 2:
                domain = hr['email'].split('@')[1]
                hr['company'] = domain.split('.')[0].replace('-', ' ').replace('_', ' ').title()
            
            cleaned_hr_details.append(hr)
        
        print(f"Extracted {len(cleaned_hr_details)} HR contacts:")
        for hr in cleaned_hr_details[:5]:  # Show first 5 as examples
            print(f"  - {hr['name']} ({hr['title']} at {hr['company']}): {hr['email']}")
        
        if len(cleaned_hr_details) > 5:
            print(f"  ... and {len(cleaned_hr_details) - 5} more")
            
        return cleaned_hr_details

def send_email(hr_detail, sender_email, sender_password, sender_name, sender_mobile, resume_path):
    """
    Send personalized email to HR with resume attachment.
    """
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = hr_detail['email']
    msg['Subject'] = f"Job Application for AI/ML Engineer Position at {hr_detail['company']}"
    
    
    body = f"""Dear {hr_detail['name']},

I am writing to express my interest in the AI/ML Engineer role at {hr_detail['company']}. Having a solid foundation in Artificial Intelligence and Machine Learning, and hands-on experience in Predictive Modeling, Data Analysis and Deep Learning, I am sure that I can be of positive value to your team.

I have a Bachelor of Engineering in Artificial Intelligence and Data Science, and I have also acquired practical experience through two prestigious internships:

AI Adventures – (Data Science and Machine Learning Intern) [3 Months] , where I have worked on Data Analysis, Web Scraping (BeautifulSoup), Sentiment Analysis and Predictive Modeling with the help of ML algorithms such as KNN, SVM, Decision Trees, and Random Forest.

Space Application Centre (ISRO) – (Research Intern) [5 Months], wherein I was involved in Rainfall Estimation using satellite data and used ML models (Linear Regression, Random Forest, Decision Tree) to enhance estimation accuracy.

My technical expertise includes Python, SQL, PyTorch, TensorFlow, scikit-learn, Pandas, Tableau, and Power BI, with emphasis on predictive modeling, AI automation, and deep learning. I also possess experience with data visualization and analytics tools.

I have Attached my resume below for your consideration. I would appreciate the chance to speak with you about how my abilities would benefit your company.

I eagerly await your response.

Best regards, 
{sender_name} 
Mobile Number: {sender_mobile}
"""
    
    msg.attach(MIMEText(body, 'plain'))
    
    with open(resume_path, 'rb') as file:
        resume = MIMEApplication(file.read(), Name=os.path.basename(resume_path))
    resume['Content-Disposition'] = f'attachment; filename="{os.path.basename(resume_path)}"'
    msg.attach(resume)
    
    try:
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()
        server.login(sender_email, sender_password)
        text = msg.as_string()
        server.sendmail(sender_email, hr_detail['email'], text)
        server.quit()
        print(f"Email sent successfully to {hr_detail['name']} at {hr_detail['company']}")
        return True
    except Exception as e:
        print(f"Error sending email to {hr_detail['email']}: {str(e)}")
        return False

def main():
    pdf_path = "CompanyWise HR contact (1).pdf" 
    resume_path = "Tanuj_Kaswa_resume.pdf" 
    sender_email = "tanujkaswa23@gmail.com" 
    sender_password = "segk zjjq lzfd swxu"
    sender_name = "Tanuj Kaswa"  
    sender_mobile = "+917385538010" 
    
    hr_details = extract_hr_details_from_pdf(pdf_path)

    if not hr_details:
        print("No HR contacts were found in the PDF. Please check the PDF format.")
        return
    
    confirmation = input(f"Do you want to send emails to all {len(hr_details)} HR contacts? (yes/no): ")
    if confirmation.lower() != 'yes':
        print("Operation cancelled.")
        return
    
    success_count = 0
    for hr in hr_details:
        success = send_email(hr, sender_email, sender_password, sender_name, sender_mobile, resume_path)
        if success:
            success_count += 1
    
    print(f"Email sending complete. Successfully sent {success_count} out of {len(hr_details)} emails.")

if __name__ == "__main__":
    main()
    


167453
Extracted text from PDF (first 500 characters):
.co.in
 Associate Vice President - HR
 Netcore Cloud
...
Found 1841 email addresses. Extracting surrounding data...
Extracted 1841 HR contacts:
  - Akanksha Puri (Associate Director HR at SourceFuse Technologies): akanksha.puri@sourcefuse.com
  - Akanksha Sogani (Head HR at Perennial Systems): akanksha.sogani@perennialsys.com
  - Akhil Jogiparthi (Vice President - Talent Accelerator at iB Hubs): akhil@ibhubs.co
  - Akhila Chandan (Associate Vice President Human Resources at Estuate,): akhila@estuate.com
  - Akram Mohammad (Deputy Head Head HR at Colruyt India): akram.mohammad@colruytgroup.com
  ... and 1836 more
Do you want to send emails to all 1841 HR contacts? (yes/no): yes
Email sent successfully to Akanksha Puri at SourceFuse Technologies
Email sent successfully to Akanksha Sogani at Perennial Systems
Email sent successfully to Akhil Jogiparthi at iB Hubs
Email sent successfully to Akhila Chandan at Estuate,
Email sent success

Email sent successfully to Arushi Goel at BetterPlace
Email sent successfully to Arushi Sawhney at Altezzasys Systems
Email sent successfully to Arvind Sadasivan at Eka Software Solutions
Email sent successfully to Asenath Sharon at SrinSoft Technologies
Email sent successfully to Asha Rao at Unilog
Email sent successfully to Ashish Karnik at Pavilion
Email sent successfully to Ashish Naidu at Mindgate Solutions
Email sent successfully to Ashok Manjunath at Spenmo
Email sent successfully to Ashok Putsala at SenecaGlobal
Email sent successfully to Ashok Seshadri at ObjectFrontier Software
Email sent successfully to Ashok Tripathy at BPO Convergence
Email sent successfully to Ashraf Kazi at Simplify Healthcare
Email sent successfully to Ashraf Mulla at qSEAp Infotech
Email sent successfully to Ashton Lawrie at MasterSoft ERP Solutions
Email sent successfully to Ashutosh Sinha at Tuyasmart India
Email sent successfully to Ashwani Bhargava at Decision Minds
Email sent successfully to Ashwa

Email sent successfully to Deepali at Proximity Works
Email sent successfully to Deepali Verdi at Genzeon
Email sent successfully to Deepashree V at Skience
Email sent successfully to Deepika Pandita at Appiness Interactive
Email sent successfully to Deepika Singh at Webkul
Email sent successfully to Deepthi Kesireddy at Smart IMS
Email sent successfully to Deepthi Vorem at ETG Digital
Email sent successfully to Deepti Ashar at MindCraft Software
Email sent successfully to Deepti Bathija at Orcapod
Email sent successfully to Deepti Lewis at Conviva
Email sent successfully to Deepti Mendiratta at Trell
Email sent successfully to Deepti N at ACCELQ
Email sent successfully to Deepti Sahni at Mobiloitte
Email sent successfully to Deepti Tewari at Quark Software
Email sent successfully to Deepti Waghmare at Prescient Technologies
Email sent successfully to Deo Kumar at Akal Information Systems
Email sent successfully to Devang Hindocha at StrategicERP Business Automation Solut
Email sent su

Email sent successfully to Hank Mishra at Anchanto
Email sent successfully to Happy Vachhani at Capermint Technologies
Email sent successfully to Hari Krishnan at AVASO Technology Solutions
Email sent successfully to Hari Paramatmuni at Excelra
Email sent successfully to Hari Prashanth at West Agile Labs
Email sent successfully to Harikrishna Bachala at Monarch Info Tech Services
Email sent successfully to Hariprasad Adthale at ASK Automotive
Email sent successfully to Haris Ali at TRUGlobal
Email sent successfully to Harishankar Krishnamurthi at Zumen Inc
Email sent successfully to Harishankher Selvaraj at Codingmart Technologies
Email sent successfully to Harista Jakhar at Impelsys
Email sent successfully to Harith Chambravalli at Observe.AI
Email sent successfully to Haritha Etakula at JNET Technologies
Email sent successfully to Harpreet Bali at NVISH Solutions
Email sent successfully to Harsh Khanna at Atlas
Email sent successfully to Harshada Moharil at Fresh Gravity
Email sent s

Email sent successfully to Kavitha Martin at SourceTrace
Email sent successfully to Kavitha Umasankar at Wolters Kluwer ELM Solutions
Email sent successfully to Kavya K at Mygo Consulting
Email sent successfully to Keerthi Kamasamudra at Stellapps Technologies
Email sent successfully to Keerthi Vinodh at RLabs Enterprise Services
Email sent successfully to Ketan Shetty at Writer Information
Email sent successfully to Kevin Marbaniang at Xeno
Email sent successfully to Kevin Parker at Integrass
Email sent successfully to Khandoba K at TechTree IT Systems
Email sent successfully to Khushboo Jain at Techspian
Email sent successfully to Khushboo Rathore at Techaheadcorp
Email sent successfully to Khushi Mishra at 5ireChain
Email sent successfully to Khyati Sagar at Appitsimple Infotek
Email sent successfully to Kinjal Shah at BiztechCS
Email sent successfully to Kiran Bala at UST BlueConch Technologies
Email sent successfully to Kiran Kumar at Adea Solutions
Email sent successfully to Kira

...

Email sent successfully to Nirmal Nimodiya at SCORG International Consulting
Email sent successfully to Nirmala Nayak at TechnoBind Solutions
Email sent successfully to Nirmiti Choudhari at EventBeep
Email sent successfully to Nirupa Leeladhar at Smaartt Digital Consulting
Email sent successfully to Nirupama Sridhar at Qwikcilver Solutions
Email sent successfully to Nirzari Sen at Benison Technologies
Email sent successfully to Nisha Motwani at Pratiti Technologies
Email sent successfully to Nisha Nair at Sagarsoft
Email sent successfully to Nisha Nayar at Technovert
Email sent successfully to Nisha Roy at Aabasoft
Email sent successfully to Nisha Saini at Auriga IT Consulting
Email sent successfully to Nisha Singh at Seanergy Digital
Email sent successfully to Nishant Gawand at SmartKargo
Email sent successfully to Nishant Shukla at Hoonartek
Email sent successfully to Nishita Algubelli at Fission Labs
Email sent successfully to Nishith Parikh at Krish TechnoLabs
Email sent successful

Email sent successfully to Pragatii Choudhary at Orbit Techsol
Email sent successfully to Pragya at Lirik
Email sent successfully to Pragya Parashar at Claritus Management Consulting
Email sent successfully to Prajeeth Gobi at WinWire Technologies
Email sent successfully to Prajeetha Prasad at ThoughtSpot
Email sent successfully to Prakash Balasubramanian at Payoda Technology
Email sent successfully to Prakash Kumaran at Newt Global Consulting
Email sent successfully to Prakash Nair at SunTec Business Solutions
Email sent successfully to Prakash Rc at Kriya IT
Email sent successfully to Prakyath Krishnappa at Verifaya Corporation
Email sent successfully to Pramodh Karumbaiah at Trell
Email sent successfully to Pranab Mishra at CriticalRiver
Email sent successfully to Pranali Shinde at Feat Systems
Email sent successfully to Pranay Mahadik at Precision Technologies
Email sent successfully to Pranitha Penmetsa at tyfone,
Email sent successfully to Prasaanth Subbiah at Ivy Mobility
Email 

Error sending email to priya.bagla@fullestop.com: (550, b'5.4.5 Daily user sending limit exceeded. For more information on Gmail\n5.4.5 sending limits go to\n5.4.5  https://support.google.com/a/answer/166852 d2e1a72fcca58-7362f6b75f5sm5713505b3a.44 - gsmtp')
Error sending email to priya.bhogineni@yexle.com: (550, b'5.4.5 Daily user sending limit exceeded. For more information on Gmail\n5.4.5 sending limits go to\n5.4.5  https://support.google.com/a/answer/166852 d9443c01a7336-223504c59ecsm81173085ad.123 - gsmtp')
Error sending email to priya.malhotra@daffodilsw.com: (550, b'5.4.5 Daily user sending limit exceeded. For more information on Gmail\n5.4.5 sending limits go to\n5.4.5  https://support.google.com/a/answer/166852 d9443c01a7336-223501fb1f7sm81066755ad.98 - gsmtp')
Error sending email to priya@flexasoft.com: (550, b'5.4.5 Daily user sending limit exceeded. For more information on Gmail\n5.4.5 sending limits go to\n5.4.5  https://support.google.com/a/answer/166852 98e67ed59e1d1-2f

Error sending email to raghavendra@syscloud.com: (550, b'5.4.5 Daily user sending limit exceeded. For more information on Gmail\n5.4.5 sending limits go to\n5.4.5  https://support.google.com/a/answer/166852 41be03b00d2f7-af233af8af3sm2708793a12.73 - gsmtp')
Error sending email to raghavendra@fidelisgroup.in: (550, b'5.4.5 Daily user sending limit exceeded. For more information on Gmail\n5.4.5 sending limits go to\n5.4.5  https://support.google.com/a/answer/166852 98e67ed59e1d1-2fea67a7ddesm9315044a91.27 - gsmtp')
Error sending email to rgawde@softcell.com: (550, b'5.4.5 Daily user sending limit exceeded. For more information on Gmail\n5.4.5 sending limits go to\n5.4.5  https://support.google.com/a/answer/166852 98e67ed59e1d1-2fea67a7183sm10390909a91.25 - gsmtp')
Error sending email to ragini.m@seanergydigital.com: (550, b'5.4.5 Daily user sending limit exceeded. For more information on Gmail\n5.4.5 sending limits go to\n5.4.5  https://support.google.com/a/answer/166852 d9443c01a7336-22

KeyboardInterrupt: 